<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.62
    Uninstalling yfinance-0.2.62:
      Successfully uninstalled yfinance-0.2.62
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-11 12:28:27
-------------------
qualified stocks: 90
with latest results: 29
still star stocks: 17
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  4.16 L
Current:  1.36 C
-------------------
Today PnL: 73.06 K (0.54%)
Current PnL: -13.27 L (-9.25%)
CY Booked + Current PnL: -7.19 L (-5.01%)
-------------------
Total profit:  4.13 L
Total loss:  -17.40 L
-------------------
Total Booked + Current PnL: 19.79 L (16.98%)
Total Booked PnL: 33.06 L (28.37%)
Curr Year Booked PnL: 6.08 L (4.46%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.05 C
Est FTT PnL: 68.78 L (50.41%)
Deployed:  1.17 C
Current:  1.36 C
CAGR/XIRR %: 11.18%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LICI,785.05,983.0,-2.96,H-LC,86.67,238452.0,43760.0,5341.0,1.48,22.48,2.24,25.21,34.0,8.19,1.80,31.86,XY25,NTT,INSURANCE
59,RELIANCE,1291.83,1526.0,7.25,X-LC,30.00,159314.0,18505.0,7026.0,1.61,13.14,4.41,18.13,19.0,2.63,1.20,25.79,XY25,NTT,REFINERIES
81,UJJIVANSFB,52.77,53.0,55.19,M-SC,78.89,134595.0,-7884.0,8506.0,-2.92,-5.53,6.32,0.44,241.0,-0.93,1.02,58.86,OX40N,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,3.60,M-LC,52.22,103731.0,25369.0,8662.0,-0.87,32.37,8.35,43.43,77.0,2.93,0.78,38.74,XY24,NTT,MISC
82,UNIONBANK,123.87,163.0,-0.31,M-LC,88.89,175473.0,34633.0,9862.0,0.01,24.59,5.62,31.59,88.0,3.51,1.33,52.42,XY24,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,NATIONALUM,189.63,247.44,-36.43,L-SC,68.89,101219.0,715.0,29920.0,-0.38,0.71,29.56,30.49,246.0,0.02,0.77,33.35,X5K,ATH,METALS
16,CAMPUS,294.86,393.00,NaN,NaN,87.78,167969.0,194.0,55648.0,2.22,0.12,33.13,33.28,200.0,0.00,1.27,30.04,XY24,NTT,FOOTWEAR
20,COALINDIA,406.65,512.15,8.40,L-LC,34.44,150054.0,-406.0,39449.0,1.50,-0.27,26.29,25.94,157.0,-0.01,1.13,14.50,XY25,ATH,MINING
30,HAVELLS,1588.02,2069.16,-4.62,X-LC,22.22,246480.0,-1251.0,76310.0,0.66,-0.51,30.96,30.30,50.0,-0.02,1.86,12.72,X40,ATH,ELECTRICAL
5,APOLLOHOSP,6904.43,8285.00,-16.00,H-LC,16.67,103755.0,189.0,20523.0,-0.02,0.18,19.78,20.00,32.0,0.01,0.78,14.28,X40N,BTT,HEALTHCARE


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,VIPIND,488.80,489.0,-949.34,H-SC,90.00,80896.0,-14420.0,14456.0,12.02,-15.13,17.87,0.04,151.0,-1.00,0.61,59.56,OX40N,NTT,MISC
63,SAMMAANCAP,170.35,326.0,-165.22,M-SC,58.89,79374.0,-22836.0,116227.0,4.99,-22.34,146.43,91.37,199.0,-0.20,0.60,28.85,XY25,NTT,FINANCE
84,VALIANTORG,512.64,838.0,-395.70,H-SC,100.00,125749.0,-7537.0,92136.0,4.99,-5.66,73.27,63.47,144.0,-0.08,0.95,110.79,XR,NTT,CHEMICALS
83,VAIBHAVGBL,350.21,521.0,72.45,X-SC,70.00,115118.0,-36873.0,110997.0,3.31,-24.26,96.42,48.77,60.0,-0.33,0.87,32.23,XR,NTT,JEWELLERY
57,RAJESHEXPO,517.65,518.0,1932.83,M-SC,62.22,55470.0,-81707.0,81802.0,3.30,-59.56,147.47,0.07,233.0,-1.00,0.42,37.96,OX40N,NTT,JEWELLERY


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,UJJIVANSFB,52.77,53.00,55.19,M-SC,78.89,134595.0,-7884.0,8506.0,-2.92,-5.53,6.32,0.44,241.0,-0.93,1.02,58.86,OX40N,NTT,BANKS
1,ABB,5551.76,7934.00,-33.55,H-LC,54.44,237354.0,20835.0,72061.0,-1.33,9.62,30.36,42.91,27.0,0.29,1.79,24.07,AR,NTT,ELECTRICAL
37,ICICIGI,1839.64,2260.25,-12.23,H-MC,48.89,144455.0,8322.0,22809.0,-1.29,6.11,15.79,22.86,33.0,0.36,1.09,19.50,X40,ATH,INSURANCE
56,QUESS,711.66,986.00,NaN,NaN,1.11,69303.0,-86551.0,146631.0,-1.26,-55.53,211.58,38.55,190.0,-0.59,0.52,6.12,XY24,NTT,MISC
0,5PAISA,564.75,565.00,158.41,H-SC,73.33,139351.0,-47581.0,47658.0,-1.02,-25.45,34.20,0.04,148.0,-1.00,1.05,33.31,OX40N,NTT,FINANCE


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,UJJIVANSFB,52.77,53.0,55.19,M-SC,78.89,134595.0,-7884.0,8506.0,-2.92,-5.53,6.32,0.44,241.0,-0.93,1.02,58.86,OX40N,NTT,BANKS
73,SYMPHONY,1306.42,1306.0,-26.33,M-SC,4.44,156833.0,-14308.0,14256.0,1.05,-8.36,9.09,-0.03,189.0,-1.00,1.19,12.72,OX40N,NTT,DURABLES
75,TATAELXSI,7332.28,7332.0,-11.78,X-MC,55.56,92554.0,-10098.0,10098.0,-0.29,-9.84,10.91,-0.00,55.0,-1.00,0.70,40.69,OX40N,NTT,IT
86,VIPIND,488.80,489.0,-949.34,H-SC,90.00,80896.0,-14420.0,14456.0,12.02,-15.13,17.87,0.04,151.0,-1.00,0.61,59.56,OX40N,NTT,MISC
47,KANSAINER,299.63,340.0,-67.31,H-SC,8.89,227340.0,-42327.0,78660.0,-0.49,-15.70,34.60,13.47,143.0,-0.54,1.72,13.79,XY24,NTT,PAINTS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TCS,3794.03,4998.00,-16.80,X-LC,7.78,270941.0,-24993.0,118916.0,0.31,-8.45,43.89,31.73,1.0,-0.21,2.05,8.45,X40,BTT,IT
42,INFY,1461.46,2275.00,-10.27,X-LC,21.11,287814.0,26213.0,119414.0,0.75,10.02,41.49,55.67,2.0,0.22,2.18,16.74,X40,BTT,IT
51,LTIM,5564.16,7197.33,0.70,X-LC,57.78,193680.0,-6630.0,65425.0,-0.09,-3.31,33.78,29.35,3.0,-0.10,1.46,33.80,X200,ATH,IT
6,ASIANPAINT,2961.56,4250.00,-24.04,X-LC,6.67,151150.0,-50236.0,137849.0,0.18,-24.95,91.20,43.51,5.0,-0.36,1.14,5.22,X40,BTT,PAINTS
34,HINDUNILVR,2413.81,2723.00,-14.85,X-LC,23.33,267546.0,-2801.0,37430.0,-0.27,-1.04,13.99,12.81,6.0,-0.07,2.02,10.73,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,UNIONBANK,123.87,163.0,-0.31,M-LC,88.89,175473.0,34633.0,9862.0,0.01,24.59,5.62,31.59,88.0,3.51,1.33,52.42,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,3.60,M-LC,52.22,103731.0,25369.0,8662.0,-0.87,32.37,8.35,43.43,77.0,2.93,0.78,38.74,XY24,NTT,MISC
50,LICI,785.05,983.0,-2.96,H-LC,86.67,238452.0,43760.0,5341.0,1.48,22.48,2.24,25.21,34.0,8.19,1.80,31.86,XY25,NTT,INSURANCE


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,130.64,228.00,103.46,M-MC,71.11,230920.0,5958.0,161690.0,0.88,2.65,70.02,74.53,183.0,0.04,1.75,34.07,XY24,BTT,STEEL
35,HINDZINC,514.80,744.74,43.14,M-LC,92.22,107260.0,4300.0,41692.0,0.42,4.18,38.87,44.67,68.0,0.10,0.81,38.42,X5K,ATH,METALS
17,CAMS,3643.00,5250.99,7.33,H-SC,84.44,120562.0,18558.0,26463.0,1.05,18.19,21.95,44.14,125.0,0.70,0.91,37.39,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,3.60,M-LC,52.22,103731.0,25369.0,8662.0,-0.87,32.37,8.35,43.43,77.0,2.93,0.78,38.74,XY24,NTT,MISC
32,HEROMOTOCO,4311.81,6039.03,1.90,H-MC,65.56,155768.0,4855.0,55594.0,1.63,3.22,35.69,40.06,101.0,0.09,1.18,27.41,AR,ATH,AUTO


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,WHIRLPOOL,1167.49,2270.00,-32.90,M-SC,43.33,195642.0,24021.0,138045.0,0.97,14.00,70.56,94.43,191.0,0.17,1.48,44.40,XR,NTT,DURABLES
11,BANKINDIA,116.91,190.00,-27.01,M-MC,75.56,193327.0,13519.0,98887.0,-0.46,7.52,51.15,62.52,168.0,0.14,1.46,39.13,XR,NTT,BANKS
35,HINDZINC,514.80,744.74,43.14,M-LC,92.22,107260.0,4300.0,41692.0,0.42,4.18,38.87,44.67,68.0,0.10,0.81,38.42,X5K,ATH,METALS
32,HEROMOTOCO,4311.81,6039.03,1.90,H-MC,65.56,155768.0,4855.0,55594.0,1.63,3.22,35.69,40.06,101.0,0.09,1.18,27.41,AR,ATH,AUTO
89,WIPRO,243.46,420.00,-7.44,M-LC,14.44,158974.0,8029.0,101425.0,0.60,5.32,63.80,72.51,70.0,0.08,1.20,9.86,XR,NTT,IT


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,59.14,80.00,-51.71,L-SC,94.44,71635.0,-41914.0,81965.0,2.98,-36.91,114.42,35.27,260.0,-0.51,0.54,75.33,XR,NTT,HOTELS
4,ALKYLAMINE,2347.98,4567.52,-7.04,X-SC,83.33,86529.0,-14434.0,109875.0,-0.07,-14.30,126.98,94.53,59.0,-0.13,0.65,28.29,SR,ATH,CHEMICALS
52,MASFIN,326.60,399.50,-19.60,H-SC,72.22,91560.0,-6420.0,28292.0,1.11,-6.55,30.90,22.32,138.0,-0.23,0.69,32.79,XR,ATH,FINANCE
3,ADANIPORTS,1103.69,1583.00,3.60,M-LC,52.22,103731.0,25369.0,8662.0,-0.87,32.37,8.35,43.43,77.0,2.93,0.78,38.74,XY24,NTT,MISC
35,HINDZINC,514.80,744.74,43.14,M-LC,92.22,107260.0,4300.0,41692.0,0.42,4.18,38.87,44.67,68.0,0.10,0.81,38.42,X5K,ATH,METALS


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,VALIANTORG,512.64,838.00,-395.70,H-SC,100.00,125749.0,-7537.0,92136.0,4.99,-5.66,73.27,63.47,144.0,-0.08,0.95,110.79,XR,NTT,CHEMICALS
7,ASIANTILES,75.41,137.00,7227.78,M-SC,98.89,80412.0,-13398.0,90013.0,1.91,-14.28,111.94,81.67,235.0,-0.15,0.61,55.72,XR,NTT,CERAMICS
46,JPPOWER,18.73,26.20,-31.29,L-SC,97.78,139495.0,-2347.0,58923.0,2.39,-1.65,42.24,39.88,256.0,-0.04,1.05,41.37,XY24,NTT,POWER
74,TANLA,917.30,1963.11,-32.58,H-SC,96.67,165448.0,-73967.0,346928.0,0.76,-30.89,209.69,114.01,127.0,-0.21,1.25,51.74,AR,ATH,IT
78,TITAGARH,1097.23,1548.00,-5.82,H-SC,95.56,165428.0,-26587.0,105477.0,0.14,-13.85,63.76,41.08,152.0,-0.25,1.25,40.01,XY24,NTT,ENGINEERING


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.78
1,25,43.62
2,50,72.52


In [21]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     32.37
XY25     15.45
XR       11.08
X40      10.96
OX40N     8.59
X40N      8.46
AR        8.05
X5K       2.38
X200      1.46
SR        1.16
Name: CurrAlloc%, dtype: float64

In [22]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    19.06
X-LC    18.20
M-SC    14.42
M-LC    10.20
H-MC     9.06
H-LC     7.84
X-MC     6.22
M-MC     5.89
L-SC     3.31
X-SC     2.84
L-LC     1.13
Name: CurrAlloc%, dtype: float64

In [23]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.20,-0.68,34.17
IT,11.19,-12.16,69.10
BANKS,8.67,-6.17,48.96
MISC,6.59,-23.70,71.95
FINANCE,6.51,-16.62,61.49
PAINTS,5.67,-16.36,39.29
INSURANCE,5.26,4.24,24.37
ELECTRICAL,5.17,-2.57,47.05
AUTO,4.78,-7.06,54.13


In [24]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2648334.0,25
XR,1093849.0,13
AR,1004422.0,9
X40,597489.0,8
XY25,485513.0,10
X40N,409399.0,8
OX40N,295251.0,11
SR,190230.0,2
X5K,88092.0,3


In [25]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1843718.0,18
M-SC,1419767.0,18
X-LC,804306.0,11
M-MC,565399.0,5
H-MC,527443.0,7
M-LC,365472.0,8
X-MC,321560.0,5
L-SC,290457.0,5
X-SC,257604.0,4


In [26]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,878828.0,6
M-SC,XY24,567142.0,5
H-SC,AR,525997.0,3
X-LC,X40,452489.0,4
M-MC,XY24,381402.0,3
M-SC,XR,339293.0,4
H-MC,XY24,216125.0,2
M-LC,XY24,194488.0,4
H-SC,XR,192162.0,3


In [27]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 52.0 seconds
